<a href="https://colab.research.google.com/github/hasnaa19/Cat-Vs-Dog/blob/main/Sports_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from zipfile import ZipFile
from google.colab import files

import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt

import cv2
from PIL import Image

import os
from tqdm import tqdm


!pip install tflearn
import tflearn
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.core.framework.types_pb2 import DataType
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
 def PrepareDirectories():
    Parent = "/content/drive/MyDrive"
    SubParent = os.path.join(Parent, '/NN_Project')
    os.mkdir(SubParent)

    with ZipFile(r"/content/drive/MyDrive/NN_Project/NN Dataset.zip", 'r') as zO:
      zO.extractall(SubParent)
      zO.close()

    Train = os.path.join(SubParent, 'Train')
    Test = os.path.join(SubParent, 'Test')

    return Train, Test, SubParent


In [ ]:
Train, Test, Folder = PrepareDirectories()

FileExistsError: ignored

In [ ]:
Train = "/content/drive/MyDrive/NN_Project/Train"
Test = "/content/drive/MyDrive/NN_Project/Test"
Folder = "/content/drive/MyDrive/NN_Project"
ImgSize = 224
Classes = ["Basketball", "Football", "Rowing", "Swimming", "Tennis", "Yoga"]

In [ ]:
def ConvertJPG(T, img):
  Path = os.path.join(T, img)
  i = img.split('.')
  Format = i[-1]
  ImageName = i[0]
  if Format == "png":
    converted = Image.open(Path).convert('RGB')
    newPath = os.path.join(T, ImageName + ".jpg")
    converted.save(newPath)
    os.remove(Path)  # To remove png file
    return newPath
  else:
    return Path

In [ ]:
def Label(img):
  l = img.split('_')
  index = Classes.index(l[0])
  Label = np.zeros(6)
  Label[index] = 1
  return Label

In [ ]:
trainAug = ImageDataGenerator(
rescale= 1./ 255,
rotation_range = 40,
horizontal_flip=True,
fill_mode='nearest')

for img in tqdm(os.listdir(Train)):
  i = img.split('_')
  if(len(i) == 2):
    path = ConvertJPG(Train,img)
    imgName = i[0]
    aug = load_img(path)
    aug_array = img_to_array(aug)
    aug_array = aug_array.reshape((1,) + aug_array.shape)

    count = 4
    for batch in trainAug.flow(aug_array, batch_size = 1, save_to_dir= Train , save_prefix= imgName + "_Augmented" , save_format = 'jpg'):
      count -=1

      if count == 0:
        break

In [ ]:
def Normalization(image):
    norm = np.zeros((ImgSize, ImgSize))
    norm_image = cv2.normalize(image,norm,0,255,cv2.NORM_MINMAX)
    return norm_image

In [ ]:
def testShuffle(training):
  for s in training[:10]:
    print(s[1])

In [ ]:
def Create_TrainingData():
  training  = []
  for img in tqdm(os.listdir(Train)):
    path = os.path.join(Train,img)
    try:
      image = cv2.imread(path, cv2.IMREAD_COLOR)
      image = cv2.resize(image, (ImgSize, ImgSize))
      image = Normalization(image)
      img_array = np.array(image)
      training.append([img_array, Label(img)])
    except Exception as e:
      print(e)
      pass

  shuffle(training)
  np.save(os.path.join(Folder, 'train_data.npy'), training) # To be saved in the folder created for project
  return training

In [ ]:
def Create_TestData():
  testing  = []
  for img in tqdm(os.listdir(Test)):
    path = os.path.join(Test,img)
    try:
      image= cv2.imread(path, cv2.IMREAD_COLOR)
      image = cv2.resize(image, (ImgSize, ImgSize))
      image = Normalization(image)
      img_array = np.array(image)
      testing.append([img_array, img])
    except Exception as e:
          print(e)
          pass
  np.save(os.path.join(Folder, 'test_data.npy'), testing)
  return testing

In [ ]:
if (os.path.exists('train_data.npy')):
    train_data = np.load('train_data.npy', allow_pickle=True) # Boolean value that allows or disallows saving the array using Python pickles.
else:
    train_data = Create_TrainingData()

if (os.path.exists('test_data.npy')):
    test_data =np.load('test_data.npy',allow_pickle=True)
else:
    test_data = Create_TestData()

print(len(train_data))
print(len(test_data))


100%|██████████| 688/688 [00:12<00:00, 53.64it/s]


7985
688


In [ ]:
def split_test_train(train, test):
    imgNames = []

    X_train = np.array([i[0] for i in train]).reshape(-1, ImgSize, ImgSize, 3)
    Y_train =np.array([i[1] for i in train])

    X_test = np.array([i[0] for i in test]).reshape(-1, ImgSize, ImgSize, 3)
    imgNames = np.array([i[1] for i in test])
    return X_train, Y_train, X_test, imgNames

X_train, Y_train, X_test, imgNames = split_test_train(train_data, test_data)
print(X_train.shape)
print(X_test.shape)

(7985, 224, 224, 3)
(688, 224, 224, 3)


In [ ]:
def LeNet():
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),activation="tanh",input_shape=(ImgSize,ImgSize,3)))
    model.add(MaxPooling2D(pool_size=(6,6)))

    model.add(Conv2D(filters=16,kernel_size=(5,5),activation="tanh"))
    model.add(MaxPooling2D(pool_size=(5,5)))

    model.add(Conv2D(filters=120,kernel_size=(5,5),activation="tanh"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(84,activation="tanh"))
    model.add(Dropout(rate=0.3))

    model.add(Dense(6,activation="softmax"))
    return model

LeNet = LeNet()
LeNet.summary()
LeNet.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
#LeNet.fit(X_train,Y_train, epochs=50)

if (os.path.exists('LeNet_model.tfl.meta')):
    LeNet.load('./LeNet_model.tfl')
else:
    LeNet.fit({'input': X_train}, {'targets': Y_train}, n_epoch=50, #input, target, n_epochs are mandatory
              snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    LeNet.save('LeNet_model.tf1')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 220, 220, 32)      2432      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 16)        12816     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 6, 6, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 2, 2, 120)         48120     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 1, 1, 120)       

NameError: ignored

In [ ]:
prediction = LeNet.predict(X_test)
classified = np.argmax(prediction, axis = 1)
print(classified)

df = pd.DataFrame({"image_name": imgNames, "label": classified})
df.to_csv("LeNet.csv", index=False)
files.download("LeNet.csv")

LeNet.save_weights("LeNet.h5")

In [ ]:
def CNN16():
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(ImgSize,ImgSize,3)))
    model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.5))
    model.add(Flatten())

    model.add(Dense(32,activation="relu"))
    model.add(Dropout(rate=0.3))
    model.add(Dense(32,activation="relu"))
    model.add(Dense(6,activation="softmax"))
    return model

CNN16 = CNN16()
CNN16.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
#CNN16.fit(X_train,Y_train, epochs=50)

if (os.path.exists('CNN16_model.tfl.meta')):
    CNN16.load('./CNN16_model.tfl')
else:
    CNN16.fit({'input': X_train}, {'targets': Y_train}, n_epoch=50, #input, target, n_epochs are mandatory
              snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    CNN16.save('CNN16_model.tf1')

In [ ]:
prediction = CNN16.predict(X_test)
classified = np.argmax(prediction, axis = 1)
print(classified)

df = pd.DataFrame({"image_name": imgNames, "label": classified})
df.to_csv("CNN16.csv", index=False)
files.download("CNN16.csv")

CNN16.save_weights("CNN16.h5")

In [ ]:
def AlexNet():
  inp = layers.Input((ImgSize, ImgSize, 3))
  x = layers.Conv2D(96, 11, 4, activation='relu')(inp)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(256, 5, 1, activation='relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(256, 3, 1, activation='relu')(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(6, activation='sigmoid')(x)

  model = Model(inputs=inp, outputs=x)
  return model

AlexNet = AlexNet()
AlexNet.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
#AlexNet.fit(X_train,Y_train, epochs=50)

if (os.path.exists('AlexNet_model.tfl.meta')):
    AlexNet.load('./AlexNet_model.tfl')
else:
    AlexNet.fit({'input': X_train}, {'targets': Y_train}, n_epoch=50, #input, target, n_epochs are mandatory
              snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    AlexNet.save('AlexNet_model.tf1')

In [ ]:
prediction = AlexNet.predict(X_test)
classified = np.argmax(prediction, axis = 1)
print(classified)

df = pd.DataFrame({"image_name": imgNames, "label": classified})
df.to_csv("AlexNet.csv", index=False)
files.download("AlexNet.csv")

AlexNet.save_weights("CNN16.h5")

In [ ]:
def CNN32():
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(ImgSize, ImgSize, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(rate=0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, tf.nn.softmax)
    ])
    return model

CNN32 = CNN32()
CNN32.summary()
CNN32.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
#CNN32.fit(X_train,Y_train, epochs=100)

if (os.path.exists('CNN32_model.tfl.meta')):
    CNN32.load('./CNN32_model.tfl')
else:
    CNN32.fit({'input': X_train}, {'targets': Y_train}, n_epoch=50, #input, target, n_epochs are mandatory
              snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    CNN32.save('CNN32_model.tf1')

In [ ]:
prediction = CNN32.predict(X_test)
classified = np.argmax(prediction, axis = 1)
print(classified)

df = pd.DataFrame({"image_name": imgNames, "label": classified})
df.to_csv("CNN32.csv", index=False)
files.download("CNN32.csv")

CNN32.save_weights("CNN32.h5")

In [ ]:
def LeNet_Testing():
  LeNet.load_weights("LeNet.h5")
  LeNet.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
  LeNet.fit(X_train,Y_train, epochs=100)

def CNN16_Testing():
  CNN16.load_weights("CNN16.h5")
  CNN16.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
  CNN16.fit(X_train,Y_train, epochs=100)

def AlexNet_Testing():
  AlexNet.save_weights("AlexNet.h5")
  AlexNet.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
  AlexNet.fit(X_train,Y_train, epochs=100)

def CNN32_Testing():
  CNN32.load_weights("CNN32.h5")
  CNN32.compile(optimizer='adamax',loss = 'categorical_crossentropy',metrics=['accuracy'])
  CNN32.fit(X_train,Y_train, epochs=100)

CNN16_Testing()
# LeNet_Testing()
# AlexNet_Testing()
# CNN32_Testing()